# Crime Analysis and Prediction

This file contains a set of experiments for analyzing and extrapolating data regarding crimes.

## 1. Data Loading and Sanitization

In [24]:
import csv, sys
import matplotlib.pyplot as plt
from helper import *
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

def chicago_from(row, params=None):
    '''
    Method which takes a row from the chichago data and returns a Crime representing that row
    '''
    city = "CHICAGO"
    date_intermediate = row[3].split(" ")
    date = date_intermediate[0]
    day = get_day_of_week(date)
    time = date_intermediate[1] + date_intermediate[2]
    time_of_day = get_time_of_day(time)
    crime = row[6]
    location = (float(row[-2][2:]), float(row[-1][0:-2]))
    return Crime(city, date, day, time, time_of_day, crime, None, None, None, location)
     

config = {
    "root_data": "data/",
    "LA_crime": "crime-in-los-angeles/Crime_Data_2010_2017.csv",
    "CH_crime": "crimes-in-chicago/Chicago_Crimes_2012_to_2017.csv",
    #"LA_schema": {'date': 2, 'time': 3, 'crime': 8, 'victim_age': 10, 'victim_sex': 11, 'weapon': 16, 'location': 25},
    "CH_schema": chicago_from,
    "CH_condensed_crime_encoding": {
        'KIDNAPPING': 0,
        'HUMAN TRAFFICKING': 0,
        'OFFSENSE INVOLVING CHILDREN': 0,
        'ROBBERY': 1,
        'BURGLARY': 1,
        'THEFT': 1,
        'MOTOR VEHICLE THEFT': 1,
        'BATTERY': 2,
        'ASSAULT': 2,
        'HOMICIDE': 2,
        'CRIM SEXUAL ASSAULT': 2,
        'SEX OFFENSE': 2,
        'NARCOTICS': 3,
        'OTHER NARCOTIC VIOLATION': 3,
        'PUBLIC PEACE VIOLATION': 4,
        'INTERFERENCE WITH PUBLIC OFFICER': 4,
        'OBSCENITY': 4,
        'PUBLIC INDECENCY': 4,
        'INTIMIDATION': 4,
        'STALKING': 4,
        'CRIMINAL TRESPASS': 4,
        'CRIMINAL DAMAGE': 5,
        'ARSON': 5,
        'NON - CRIMINAL': 6,
        'NON-CRIMINAL': 6,
        'NON-CRIMINAL (SUBJECT SPECIFIED)': 6,
        'OTHER OFFENSE': 6,
        'DECEPTIVE PRACTICE': 6,
        'CONCEALED CARRY LICENSE VIOLATION': 7,
        'WEAPONS VIOLATION': 7,
        'PROSTITUTION': 8,
        'LIQUOR LAW VIOLATION': 8,
        'GAMBLING': 8
    },
    "CH_condensed_crime_decoding": [
        "KIDNAPPING / CHILDREN",
        "ROBBERY/BURGLARY/THEFT",
        "ASSAULT/VIOLENCE",
        "NARCOTICS",
        "PUBLIC-RELATED CRIME",
        "DAMAGE/ARSON",
        "OTHER/NON-CRIMINAL",
        "WEAPON-RELATED",
        "PROHIBITIVE CRIME"
    ],
}

def convert_crime_class_to_condensed_integer_CH(c):
    index = config["CH_condensed_crime_encoding"].get(c, 6)
    result = np.zeros(9)
    result[index] = 1
    return result
    
def convert_one_hot_encoding_to_crime_class_CH(one_hot_encoding):
    index = np.argmax(one_hot_encoding > 0)
    return config["CH_condensed_crime_decoding"][index]

def save_full_crime_encoding(encoding):
    config["CH_full_encoding"] = encoding

def convert_crime_class_to_full_integer_CH(c):
    index = config["CH_full_encoding"].index(c)
    result = np.zeros(len(config["CH_full_encoding"]))
    result[index] = 1
    return result

def convert_one_hot_full_encoding_to_crime_class_CH(one_hot_encoding):
    index = np.argmax(one_hot_encoding > 0)
    return config["CH_full_encoding"][index]

def get_decoder_from(comp_list):
    
    def decode_result(result):
        inc = 0
        results = []
        if "day" in comp_list:
            day = np.argmax(result[inc:inc+7] > 0)
            results.append(["SUN","MON","TUE","WED", "THU", "FRI", "SAT"][day])
            inc += 7
        if "time" in comp_list:
            time = np.argmax(result[inc:inc+4] > 0)
            results.append(["MORNING","AFTERNOON","EVENING","LATE NIGHT"][time])
            inc += 4
        if "location" in comp_list:
            latlong = result[inc:inc+2]
            results.append(latlong)
            inc += 2
        if "crime condensed" in comp_list:
            crime = result[inc:inc+9]
            results.append(convert_one_hot_encoding_to_crime_class_CH(crime))
            inc += 9
        if "crime full" in comp_list:
            crime = result[inc:inc+len(config["CH_crime_encoding"])]
            results.append(convert_one_hot_full_encoding_to_crime_class_CH(crime))
            inc += len(config["CH_crime_encoding"])
            
        return results
    return decode_result

# LA: ['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred', 'Area ID', 'Area Name', 'Reporting District', 'Crime Code', 'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex', 'Victim Descent', 'Premise Code', 'Premise Description', 'Weapon Used Code', 'Weapon Description', 'Status Code', 'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3', 'Crime Code 4', 'Address', 'Cross Street', 'Location']

class Crime:
    
    def __init__(self, city, date, day, raw_time, time_of_day, crime, victim_age, victim_sex, weapon, location):
        self.city = city
        self.date = date
        self.day = day
        self.raw_time = raw_time
        self.time_of_day = time_of_day
        self.crime = crime
        self.victim_age = victim_age
        self.victim_sex = victim_sex
        self.weapon = weapon
        self.location = location
        
    def get_specified_vector(self, comp_list):
        '''
        Returns a feature vector representing this crime.
        '''
        feature = np.array([])
        if "day" in comp_list:
            feature = np.concatenate((feature, self.day[0]))
        if "time" in comp_list:
            feature = np.concatenate((feature, self.time_of_day[0]))
        if "location" in comp_list:
            others = np.array([self.location[0], self.location[1]])
            feature = np.concatenate((feature, others))
        if "crime condensed" in comp_list:
            feature = np.concatenate((feature, convert_crime_class_to_condensed_integer_CH(self.crime)))
        if "crime full" in comp_list:
            feature = np.concatenate((feature, convert_crime_class_to_full_integer_CH(self.crime)))
        
        return feature
        
        
    def __str__(self):
        return self.crime + " in the " + str(self.time_of_day) + " on " + str(self.day)
        

def load_data(force_refresh=False):
    
    #all_data = {"LA": [], "CH": []}
    all_data = {"CH": []}
    
    for city in all_data.keys():
        data_file = config["root_data"] + config[city + "_crime"]
        parse_from = config[city + "_schema"]
        with open(data_file) as csvfile:
            content = csvfile.readlines()
            content = [x.strip() for x in content]
            del content[0] # Remove the header
            all_data[city].append([])
            
            # Save the number of possible rows available
            all_data[city].append(len(content))
            
            count = 0
            for row in content[0:600]:
                try:
                    d = row.split(",")
                    new_crime = parse_from(d)
                    all_data[city][0].append(new_crime)
                except:
                    #print("Unexpected error:", sys.exc_info()[0])
                    #raise
                    count = count + 1
                    
            # Save the number of errored rows
            all_data[city].append(count)
            
    return all_data

results = load_data()
print("Finished loading data")

Finished loading data


In [25]:
def get_workable_data(data, X_comps, Y_comps):
    '''
    Returns a matrix where rows are feature vectors of 
    [1-hot day vector, 1-hot time vector, crime, crime encoding (9 DIGITS LONG), latitude, longitude,]
    Returned as strings due to the crime being a string. Age, sex, and weapon are omitted for now
    since the Chicago data set does not include those.
    '''
    X_matrix = np.array([i.get_specified_vector(X_comps) for i in data])
    Y_matrix = np.array([i.get_specified_vector(Y_comps) for i in data])
    X_decoder = get_decoder_from(X_comps)
    Y_decoder = get_decoder_from(Y_comps)
    return (X_matrix, Y_matrix, X_decoder, Y_decoder)

# ATTENTION: HERE IS HOW EVERYTHING WORKS!!! ------------
#   1) Load the data using the cell above
#   2) (Optional) If using non-condensed crimes (i.e no combination), pass
#      a list of all crimes to save_full_crime_encoding()
#   3) Call get_workable_data with the data from 1), pass in a list of features
#      you want to include for X and target Y. Possible values are:
#         "day" "time" "location" "crime condensed" "crime full"
#   4) You will receive the encoded X matrix, and encoded Y matrix, and two methods
#      for decoding the X and Y features

# Use this next line for enabling the full crime set
save_full_crime_encoding(list(set([i.crime for i in results["CH"][0]])))

X_features = ["day", "time", "location"]
Y_features = ["crime condensed"]
X, Y, X_decoder, Y_decoder = get_workable_data(results["CH"][0], X_features, Y_features)
print("Featurization achieved")
print(X)
print(X.shape)
print(Y)
print(X_decoder(X[0]))
print(Y_decoder(Y[0]))

Featurization achieved
[[  0.           1.           0.         ...,   0.          41.86407316
  -87.70681861]
 [  0.           1.           0.         ...,   0.          41.78292153
  -87.60436317]
 [  0.           1.           0.         ...,   0.          41.89490828
  -87.75837196]
 ..., 
 [  0.           0.           1.         ...,   0.          41.87889125
  -87.7239985 ]
 [  0.           0.           1.         ...,   0.          41.97852489
  -87.83658904]
 [  0.           0.           1.         ...,   0.          41.89660508
  -87.66241328]]
(570, 13)
[[ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
['MON', 'EVENING', array([ 41.86407316, -87.70681861])]
['ASSAULT/VIOLENCE']


In [50]:
def train_KNN(feature_matrix, targets, n):
    knn = KNeighborsClassifier(n_neighbors=n)
    Y = np.zeros(targets.shape[0])
    for i in range(targets.shape[0]):
        Y[i] = np.argmax(targets[i])
    knn.fit(feature_matrix,Y)
    return knn


In [51]:
def int_to_category(prediction):
    categories = ["KIDNAPPING / CHILDREN","ROBBERY/BURGLARY/THEFT","ASSAULT/VIOLENCE","NARCOTICS","PUBLIC-RELATED CRIME","DAMAGE/ARSON","OTHER/NON-CRIMINAL","WEAPON-RELATED","PROHIBITIVE CRIME"]
    return categories[int(prediction)]


In [52]:
def evaluate_KNN(X, Y):
    # Loading data
    X_train = X[1:300]
    Y_train = Y[1:300]
    X_validate = X[300:500]
    Y_validate = Y[300:500]
    X_test = X[500:600]
    Y_test = Y[500:600]
    
    #Tuning on the validation set for value of K
    K = [k for k in range(1,30)]
    best_K = 0
    max_score = 0.0
    for k in K:
        knn = train_KNN(X_train, Y_train, k)
        predictions = knn.predict(X_validate)
        correct = 0.0
        for i in range(predictions.shape[0]):
            if int_to_category(predictions[i]) == Y_decoder(Y_validate[i])[0]:
                correct += 1.0
        if correct > max_score:
            max_score = correct
            best_K = k
        print("Validation set Performance with k = " + str(k) + ":" + str(correct/predictions.shape[0]))
    
    #Evaluating on Test set with best K value
    knn = train_KNN(X_train, Y_train, best_K)
    predictions = knn.predict(X_test)
    correct = 0.0
    for i in range(predictions.shape[0]):
        if int_to_category(predictions[i]) == Y_decoder(Y_test[i])[0]:
            correct += 1.0
    if correct > max_score:
        max_score = correct
        best_K = k
    print("Test set Performance with k = " + str(best_K) + ":  " + str(correct/predictions.shape[0]))
        
evaluate_KNN(X, Y)       
        
    
    
    
    

Validation set Performance with k = 1:0.145
Validation set Performance with k = 2:0.18
Validation set Performance with k = 3:0.205
Validation set Performance with k = 4:0.235
Validation set Performance with k = 5:0.255
Validation set Performance with k = 6:0.27
Validation set Performance with k = 7:0.27
Validation set Performance with k = 8:0.265
Validation set Performance with k = 9:0.265
Validation set Performance with k = 10:0.275
Validation set Performance with k = 11:0.29
Validation set Performance with k = 12:0.26
Validation set Performance with k = 13:0.275
Validation set Performance with k = 14:0.31
Validation set Performance with k = 15:0.3
Validation set Performance with k = 16:0.315
Validation set Performance with k = 17:0.275
Validation set Performance with k = 18:0.28
Validation set Performance with k = 19:0.265
Validation set Performance with k = 20:0.27
Validation set Performance with k = 21:0.25
Validation set Performance with k = 22:0.245
Validation set Performance wit